ESQUELETO:

Este es el archivo para subir el modelo a la nube.


CÓMO FUNCIONA:

1. Tomar un modelo ya realizado.

2. Lo sube a la nube.

ASEGÚRENSE DE QUE SE COORDINAN LO NECESARIO PARA QUE LOS NOMBRES COINCIDAN (no es de chill cambiar el estándar xd).

In [2]:
## STEP 1: Set up the workspace.

import json

#GETTING MY ID:
id = open('id_CONFIDENTIAL.json', 'r')
mi = json.load(id)
my_id = mi["my_id"],

from azureml.core import Workspace

ws = Workspace.create(name="workspace",
                      subscription_id = my_id,
                      resource_group = "resource_group2",
                      location = "centralindia")

from azureml.core.model import Model

mname = "model"
registered_model = Model.register(model_path="model.pkl",
                                  model_name=mname,
                                  workspace=ws)

Cleaning up past default Resource Group Deployments on the subscription to avoid limit of 10
Deleting past Resource Group Deployment with name: DeployResourceGroup-10783a0eac
Deploying StorageAccount with name workspacstorage81803bc63.
Deploying KeyVault with name workspackeyvaultbaf36cd2.
Deploying AppInsights with name workspacinsightsbed1c8e8.
Deployed AppInsights with name workspacinsightsbed1c8e8. Took 41.57 seconds.
Deploying Workspace with name workspace.
Deployed Workspace with name workspace. Took 45.97 seconds.
Registering model model


In [9]:
################ score.py ###################

umb = open("umbral.json", "r")
umb = json.load(umb)
umbral = umb["umbral"][0]

scorepy = f"""
import json
import joblib
import numpy as np
import pandas as pd
from azureml.core.model import Model

def init():
  global model
  model_path = Model.get_model_path('{mname}')
  model = joblib.load(model_path)

def sigmoid(x):
  return [1 / (1 + np.exp(-y)) for y in x]

def run(raw_data):
  try: ## Try la predicción.
    data = json.loads(raw_data)['data'][0]
    data = pd.DataFrame(data)

    data_dummies = data.drop(["RowNumber", "CustomerId", "Surname", "Geography", "Gender", "Card Type"], axis=1) ## INCLUIR AQUÍ LO DE ZORROUNO

    result = model.predict(data_dummies).tolist()
    result_sigmoid = sigmoid(result)
    umbral = {umbral}
    result_finals = [1 if x > umbral else 0 for x in result_sigmoid]

    return json.dumps(result_finals)
  except Exception as e:
    return json.dumps(str(e))
"""

file_score = open("score.py", "w")
file_score.write(scorepy)
file_score.close()

In [10]:
## STEP 2: Inference configuration: the blueprints for the German carpenter about how to build your furniture.

from azureml.core.environment import Environment
virtual_env = Environment("env-4-housing")

from azureml.core.conda_dependencies import CondaDependencies
virtual_env.python.conda_dependencies = CondaDependencies.create(conda_packages=['pandas','scikit-learn']) ## Asegúrense de coordinarse con el departamento de modelos para incluir los paquetes correctos. Si no, BOOM!


from azureml.core.model import InferenceConfig
from azureml.core.webservice import AciWebservice
inference_config = InferenceConfig(
                                environment=virtual_env,
                                entry_script="score.py",
                                )
aci_config = AciWebservice.deploy_configuration(cpu_cores=0.5, memory_gb=1) ## ASEGÚRENSE DE ASIGNAR SUFICIENTE MADERA PARA SUS MUEBLES.

service = Model.deploy(workspace=ws,
                       name='service',
                       models=[registered_model],
                       inference_config=inference_config,
                       deployment_config=aci_config,
                       overwrite=True,
                       )

/tmp/ipykernel_54428/944137596.py:18: FutureWarning: azureml.core.model:
To leverage new model deployment capabilities, AzureML recommends using CLI/SDK v2 to deploy models as online endpoint, 
please refer to respective documentations 
https://docs.microsoft.com/azure/machine-learning/how-to-deploy-managed-online-endpoints /
https://docs.microsoft.com/azure/machine-learning/how-to-attach-kubernetes-anywhere 
For more information on migration, see https://aka.ms/acimoemigration 
To disable CLI/SDK v1 deprecation warning set AZUREML_LOG_DEPRECATION_WARNING_ENABLED to 'False'
  service = Model.deploy(workspace=ws,


In [11]:
service.wait_for_deployment()

scoring_uri = service.scoring_uri

scoreuri = json.dumps({"URI": [scoring_uri]})
file = open("uri.json", "w")
file.write(scoreuri)
file.close()

ACI service creation operation finished, operation "Succeeded"
